In [1]:
words = [w.lower() for w in open('Indian_Names.txt','r').read().splitlines()]
words[:5]
len(words)

53982

In [2]:
chars =  sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27

In [3]:
import torch
xs,ys = [],[]
for w in words[:1] :
    chr = ['<S>'] + list(w) + ['<E>']
    for c1,c2 in zip(chr,chr[1:]):
        print(c1,c2)
        xs.append(stoi[c1])
        ys.append(stoi[c2])
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs)
print(ys)

<S> a
a a
a b
b a
a n
n <E>
tensor([26,  0,  0,  1,  0, 13])
tensor([ 0,  0,  1,  0, 13, 27])


In [4]:
import torch.nn.functional as F
xenc,yenc = F.one_hot(xs, num_classes=28).float(), F.one_hot(ys, num_classes=28).float()
print(xenc.shape, yenc.shape)

torch.Size([6, 28]) torch.Size([6, 28])


In [5]:
gen = torch.Generator().manual_seed(21474836478) 
W =  torch.randn((28, 28))  # weight matrix
counts = (xenc @ W).exp()
prob =  counts / counts.sum(1, keepdim=True)
print(prob)

tensor([[0.0046, 0.0269, 0.0468, 0.0482, 0.0178, 0.0108, 0.0315, 0.0058, 0.0280,
         0.0036, 0.0150, 0.0070, 0.0742, 0.0141, 0.1679, 0.0028, 0.0059, 0.0127,
         0.0295, 0.1020, 0.0323, 0.1004, 0.0063, 0.0888, 0.0437, 0.0236, 0.0210,
         0.0289],
        [0.0334, 0.0471, 0.0290, 0.1841, 0.0045, 0.0047, 0.0160, 0.0346, 0.1300,
         0.0082, 0.0081, 0.0159, 0.0054, 0.0068, 0.0053, 0.0034, 0.0178, 0.0101,
         0.0373, 0.0184, 0.0180, 0.0191, 0.0322, 0.0641, 0.1299, 0.0221, 0.0597,
         0.0347],
        [0.0334, 0.0471, 0.0290, 0.1841, 0.0045, 0.0047, 0.0160, 0.0346, 0.1300,
         0.0082, 0.0081, 0.0159, 0.0054, 0.0068, 0.0053, 0.0034, 0.0178, 0.0101,
         0.0373, 0.0184, 0.0180, 0.0191, 0.0322, 0.0641, 0.1299, 0.0221, 0.0597,
         0.0347],
        [0.0179, 0.0228, 0.0862, 0.0049, 0.0387, 0.0049, 0.0058, 0.0136, 0.0872,
         0.0183, 0.1345, 0.0399, 0.0126, 0.0070, 0.0105, 0.1443, 0.0084, 0.0993,
         0.0240, 0.0219, 0.0466, 0.0091, 0.0142, 0.0063

In [6]:
prob[0].sum()

tensor(1.0000)

Generating the nll of a word with the randomized weights

In [7]:
itos = {i:s for s,i in stoi.items()}
nll = []
for i in range(xs.shape[0]):
    x = xs[i].item()
    y = ys[i].item()
    print(f'bigram example {i + 1}: {itos[x]} -> {itos[y]} (indexes {x},{y})')
    print('input to the neural network:', x)
    #print('output of the neural network:', prob[i])
    print('actual next character:', itos[y])
    print('probability of the next character:', prob[i, y].item())
    nll_value = -torch.log(prob[i, y])
    nll.append(nll_value)
    print('negative log likelihood:', nll_value.item())
    print('----------')
nll = torch.stack(nll)
print('average negative log likelihood:', nll.mean().item())

bigram example 1: <S> -> a (indexes 26,0)
input to the neural network: 26
actual next character: a
probability of the next character: 0.004594045225530863
negative log likelihood: 5.382994174957275
----------
bigram example 2: a -> a (indexes 0,0)
input to the neural network: 0
actual next character: a
probability of the next character: 0.033369746059179306
negative log likelihood: 3.4001054763793945
----------
bigram example 3: a -> b (indexes 0,1)
input to the neural network: 0
actual next character: b
probability of the next character: 0.04713344946503639
negative log likelihood: 3.05477237701416
----------
bigram example 4: b -> a (indexes 1,0)
input to the neural network: 1
actual next character: a
probability of the next character: 0.01789383962750435
negative log likelihood: 4.023298740386963
----------
bigram example 5: a -> n (indexes 0,13)
input to the neural network: 0
actual next character: n
probability of the next character: 0.006839754991233349
negative log likelihood: 4

In the above output the loss was very high with randomnized weights without trainign the NN

Designing a single layer Neural Net with softmax

In [8]:
# representing the loss in tensor
loss = -prob[torch.arange(6), ys].log().mean()
print(loss)

tensor(4.3290)


In [9]:
gen = torch.Generator().manual_seed(21474836478)
W = torch.randn((28, 28), generator=gen,requires_grad= True)  # weight matrix

In [10]:
#forward pass

xenc = F.one_hot(xs, num_classes=28).float()
logits = xenc @ W
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(xs.shape[0]), ys].log().mean()
print(loss) 

tensor(3.8174, grad_fn=<NegBackward0>)


In [11]:
# representing the loss in tensor
loss = -prob[torch.arange(xs.shape[0]), ys].log().mean()
print(loss)

tensor(3.8174, grad_fn=<NegBackward0>)


In [12]:
#backward pass
W.grad = None
loss.backward()

In [13]:
#W.grad

In [14]:
W.data += -0.1 * W.grad



In [15]:
xs,ys = [],[]
for w in words:
    chr = ['<S>'] + list(w) + ['<E>']
    for c1,c2 in zip(chr,chr[1:]):
        xs.append(stoi[c1])
        ys.append(stoi[c2])
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print('no. of examples:', xs.shape[0])
g = torch.Generator().manual_seed(21474836478)
W = torch.randn((28, 28), generator=g, requires_grad=True)  #weight matrix           

no. of examples: 488074


Training the neural network with gradient descent

In [16]:
lr = 20
epochs = 300
for k in range(epochs):
    
    xenc = F.one_hot(xs, num_classes=28).float()
    logits = xenc @ W
    counts = logits.exp()
    prob = counts / counts.sum(1, keepdim=True)
    loss = -prob[torch.arange(xs.shape[0]), ys].log().mean()
    print(loss.item())
    #backward passs
    W.grad = None
    loss.backward()
    #updating the weights after backpropagation
    W.data += -lr * W.grad
    

3.6135923862457275
3.3724989891052246
3.176208972930908
3.0262224674224854
2.9123330116271973
2.825191020965576
2.758117437362671
2.7050046920776367
2.661327838897705
2.624281406402588
2.592236042022705
2.5642154216766357
2.5395736694335938
2.517829418182373
2.4985861778259277
2.4814960956573486
2.466247081756592
2.4525644779205322
2.440211772918701
2.428990125656128
2.418738603591919
2.4093258380889893
2.4006450176239014
2.392608165740967
2.3851423263549805
2.378185272216797
2.37168288230896
2.3655893802642822
2.359863758087158
2.354470729827881
2.349379301071167
2.34456205368042
2.3399951457977295
2.3356566429138184
2.3315281867980957
2.3275933265686035
2.3238375186920166
2.3202476501464844
2.3168118000030518
2.3135204315185547
2.310363531112671
2.30733323097229
2.304421901702881
2.3016226291656494
2.298928737640381
2.2963356971740723
2.293837547302246
2.2914295196533203
2.28910756111145
2.2868669033050537
2.2847044467926025
2.282615900039673
2.2805988788604736
2.2786495685577393
2.2

In [17]:
gen= torch.Generator().manual_seed(21474836478)

for i in range(20):
    out = []
    ix = 26 # start with the first character
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=28).float()
        logits = xenc @ W
        counts = logits.exp()
        prob = counts / counts.sum(1, keepdim=True)
        ix = torch.multinomial(prob, num_samples=1, generator=gen).item()
        out.append(itos[ix])
        if ix == stoi['<E>']:
            break
    print(''.join(out))

mik<E>
jlori<E>
abamathanaeraya<E>
ka<E>
savithrnanfrararun<E>
s<E>
maanek<E>
pripun<E>
ipcn<E>
ishiya<E>
kahel<E>
kalukhyarullaanth<E>
veiraanyanulln<E>
rdhukodhak<S>n<E>
sniqununi<E>
shiandhiniragal<E>
kranjwaya<E>
rabal<E>
n<E>
karai<E>
